In [1]:
import os, cv2
import random
from sklearn.metrics import confusion_matrix
import numpy as np
from PIL import Image , ImageOps  
from numpy import asarray
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten,GlobalMaxPool2D,Dropout
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.applications import Xception

In [3]:
img_size=224
datadir="data/tm/test"
training_data=[]
categories=["Male","Female"]
for category in categories:
    path=os.path.join(datadir,category)
    class_num=categories.index(category)
    for img in os.listdir(path):
        try:
            image = cv2.imread(os.path.join(path,img))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (img_size, img_size))
            training_data.append([image, class_num])
        except Exception as e:
            pass
random.shuffle(training_data)


In [4]:
x_test=[]
y_test=[]
for image,gender in training_data:
    x_test.append(image)
    y_test.append(gender)
X_train=[]
for i in range(len(x_test)):
    img_array=asarray(x_test[i])
    X_train.append(img_array)
x_test=X_train


In [5]:
x_test=np.array(x_test)
x_test=x_test.reshape(-1,224,224,3)

In [6]:
IMAGE_SIZE = [224, 224]

In [7]:
vgg =VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [8]:
for layer in vgg.layers:
  layer.trainable = False

In [9]:
#fc1 = vgg.layers[-3]
#fc2 = vgg.layers[-2]
#predictions = vgg.layers[-1]

# Create the dropout layers
#dropout1 = Dropout(0.09)
#dropout2 = Dropout(0.09)

# Reconnect the layers
#x = dropout1(fc1.output)
#x = fc2(x)
#predictors = predictions(x)
#prediction= Flatten()(predictors)
# x = Dense(1000, activation='relu')(x)
#out = Dense(2, activation='softmax')(prediction)


In [ ]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)


In [10]:
model = Model(inputs=vgg.input, outputs=out)

In [11]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory('data/tm/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3005 images belonging to 2 classes.


In [14]:
validation_set = valid_datagen.flow_from_directory('data/tm/validation',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 198 images belonging to 2 classes.


In [15]:
h = model.fit(
  training_set,
  validation_data=validation_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set)
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 94 steps, validate for 7 steps
Epoch 1/5
94/94 [==============================] - 395s 4s/step - loss: 0.5979 - accuracy: 0.7404 - val_loss: 0.3190 - val_accuracy: 0.8485
Epoch 2/5
94/94 [==============================] - 377s 4s/step - loss: 0.3250 - accuracy: 0.8506 - val_loss: 0.2953 - val_accuracy: 0.8535
Epoch 3/5
94/94 [==============================] - 411s 4s/step - loss: 0.2350 - accuracy: 0.9025 - val_loss: 0.2889 - val_accuracy: 0.8636
Epoch 4/5
94/94 [==============================] - 432s 5s/step - loss: 0.2480 - accuracy: 0.8948 - val_loss: 0.3354 - val_accuracy: 0.8434
Epoch 5/5
94/94 [==============================] - 434s 5s/step - loss: 0.1910 - accuracy: 0.9208 - val_loss: 0.3423 - val_accuracy: 0.8586


In [18]:
y_pred=model.predict(x_test)
y_pred_final=[]
for i in range(len(y_pred)):
    y_pred_final.append(y_pred[i][0])
y_pred_final=np.round(y_pred_final)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_pred_final,y_test)
model_accuracy=(cm[0][0]+cm[1][1]) / ( cm[1][0] + cm[0][1]+cm[0][0]+cm[1][1])
model_accuracy

0.927920298879203


In [19]:
model.save('gender_detection.h5')